+ $X \sim G(p)$. Then, $\mathbb{E}(X) = \frac{1}{p}$.
+ $p_X(x) = (1-p)^{x-1}p \quad (x \geq 1)$
+ Let $C$ be ceiling. Then, expectation changes as:
\begin{align*}
    \mathbb{E}(X) &= \sum_{i=1}^C xp_X(x) + (C+1) \cdot P(X \geq C+1) \\
    &= p\sum_{i=1}^C x(1-p)^{x-1} + (C+1) \cdot (1-p)^C \\
    &= p \cdot \frac{1}{p}\left[ \frac{1 - (1-p)^C}{p} - C(1-p)^C \right] + (C+1) \cdot (1-p)^C \\
    &= \frac{1 - (1-p)^C}{p} + (1-p)^C = \frac{1}{p} - \left( \frac{1}{p}-1 \right)(1-p)^C \\
    &= \frac{1}{p} - \frac{(1-p)^{C+1}}{p}
\end{align*}

+ Hence, Ceiling $C$ reduces the expectation by $\frac{(1-p)^{C+1}}{p}$, or $100*(1-p)^{C+1}$%.

In [1]:
import numpy as np
import pandas as pd
from generate_prob_chart import generate_dest_prob_chart

In [2]:
p = 0.1026
ceil = 15
mean = 0
pmf = lambda x: (1-p) ** (x-1) * p
total_prob = 0

for x in range(1, ceil+1):
    total_prob += pmf(x)
    mean += x * pmf(x)

mean += (ceil+1) * (1-p)**ceil
total_prob += (1-p)**ceil

print(f'mean(ceiling):   {mean:.4f}\nmean(estimated): {1/p-(1-p)**(ceil+1)/p:.4f}')
print(f'reduced mean:    {1/p-mean:.4f}, {1-mean*p:.1%} reduction')

mean(ceiling):   8.0222
mean(estimated): 8.0222
reduced mean:    1.7244, 17.7% reduction


In [3]:
def ceiling_reduction_analysis(starcatch=False, event_15=False, dest_prevention=False, pretty=False):
    dest_prob_chart = generate_dest_prob_chart(starcatch, event_15, dest_prevention, pretty=False)
    mean_ceiling = lambda p, ceil: 1/p - (1-p)**(ceil+1)/p
    mean_reduction_chart = pd.DataFrame(
        np.empty((5, 4)),
        columns=['original', 'ceiling', 'reduced', 'reduction %'],
        index=range(18, 23)
    )
    mean_reduction_chart.ceiling = mean_reduction_chart.ceiling.astype(int)

    for end in range(18, 23):
        dest_prob = dest_prob_chart.loc[end, 15]
        ceil = np.ceil(1/dest_prob*1.5).astype(int)
        reduced_mean = mean_ceiling(dest_prob, ceil)
        reduction_percentage = (1-dest_prob)**(ceil+1)

        mean_reduction_chart.loc[end, :] = [1/dest_prob, ceil, reduced_mean, reduction_percentage]
    
    if pretty is True:
        mean_reduction_chart['reduction %'] = mean_reduction_chart['reduction %'].apply(lambda x: f'{x:4.1%}')
        mean_reduction_chart.iloc[:, [0, 2]] = mean_reduction_chart.iloc[:, [0, 2]].applymap(lambda x: f'{x:6.4f}')
    return mean_reduction_chart

In [4]:
ceiling_reduction_analysis(pretty=True)

,original,ceiling,reduced,reduction %
18,1.8151,3,1.7413,4.1%
19,2.8483,5,2.6356,7.5%
20,4.8480,8,4.2418,12.5%
21,5.9792,9,5.0202,16.0%
22,9.7498,15,8.0238,17.7%
